In [1]:
from espnet2.tasks.asr_TruCLeS import ASRTask
from pathlib import Path
import yaml
import argparse
import torch

model_tool = ASRTask

In [2]:
import torch

print(torch.version.cuda)

11.7


In [3]:
import torchaudio

test_dataset = torchaudio.datasets.LIBRISPEECH("/home/pb_deployment/Downloads", url="test-clean", download=True)

In [4]:
import os
model_root_dir = "/home/pb_deployment/espnet/asr_inference/model_files/e_branchformer_librispeech/"
asr_config_path = os.path.join(model_root_dir, "exp/asr_train_asr_e_branchformer_raw_en_bpe5000_sp/config.yaml")
model_path = "/home/pb_deployment/espnet/asr_inference/model_files/e_branchformer_librispeech/exp/asr_train_asr_e_branchformer_raw_en_bpe5000_sp/valid.acc.ave_10best.pth"

config_file = Path(asr_config_path)
with config_file.open("r", encoding="utf-8") as f:
    args = yaml.safe_load(f)

args = argparse.Namespace(**args)
model = model_tool.build_model(args)

state_dict = torch.load(model_path, map_location='cuda:0')


model.to('cuda:0')
use_lora = getattr(args, "use_lora", False)
model.load_state_dict(state_dict, strict=not use_lora)
if any(["frontend.upstream.model" in k for k in state_dict.keys()]):
    if any(
        [
            "frontend.upstream.upstream.model" in k
            for k in dict(model.named_parameters())
                        ]
    ):
        state_dict = {
            k.replace(
                "frontend.upstream.model",
                "frontend.upstream.upstream.model",
            ): v
            for k, v in state_dict.items()
        }
        model.load_state_dict(state_dict, strict=not use_lora)

In [5]:
import yaml
import os

with open(asr_config_path, 'r') as file:
    config_data = yaml.safe_load(file)
tokens_list = config_data.get('token_list', [])
tokens_path = "tokens.txt"
with open(tokens_path, 'w') as file:
    for item in tokens_list:
        file.write(f"{item}\n")

In [6]:
from espnet2.text.build_tokenizer import build_tokenizer
from espnet2.text.token_id_converter import TokenIDConverter


def convert_text_to_tokens(text, bpe_model_path, token_list):
    tokenizer = build_tokenizer(
        token_type='bpe',
        bpemodel=bpe_model_path
    )
    tok_to_id = TokenIDConverter(
        token_list=token_list
    )
    tokens = tokenizer.text2tokens(text)
    list_of_toks = [curr[0] for curr in tokens]
    text_rep = tok_to_id.tokens2ids(list_of_toks)
    return torch.tensor(text_rep)

bpe_model_path = '/home/pb_deployment/espnet/asr_inference/model_files/e_branchformer_librispeech/data/en_token_list/bpe_unigram5000/bpe.model'
text = convert_text_to_tokens(test_dataset[0][2], bpe_model_path, tokens_path)

In [7]:
speech = test_dataset[0][0].numpy().squeeze()
speech = torch.tensor(speech)

speech = speech.unsqueeze(0)
lengths = speech.new_full([1], dtype=torch.long, fill_value=speech.size(1))

text = text
text_length = len(text)
text = text.unsqueeze(0)
text_length = torch.tensor([text_length])

In [8]:
print(speech)
print(lengths)
print(text)
print(text_length)

tensor([[0.0003, 0.0003, 0.0004,  ..., 0.0021, 0.0021, 0.0016]])
tensor([166960])
tensor([[346, 346, 346, 346, 346, 346, 242, 346, 346, 346, 270,   3, 346, 346,
          72,  22,   3, 346, 346,  72, 270, 346, 346, 346, 346, 216,  22, 346,
         346, 346, 346, 105, 346, 346, 346, 346,  48, 346, 346,  22,  48, 346]])
tensor([42])


In [9]:
DEVICE = "cuda:0"

out = model(
    speech = speech.to(DEVICE),
    speech_lengths = lengths.to(DEVICE),
    text = text.to(DEVICE),
    text_lengths = text_length
)

torch.Size([1, 43, 5000])


In [10]:
print(model.decoder.train_decoder.shape)
print(model.decoder.train_softmax.shape)
print(model.encoder.train_attention.shape)

torch.Size([1, 43, 512])
torch.Size([1, 43, 5000])
torch.Size([1, 260, 512])


In [11]:
model.decoder.train_decoder

tensor([[[  0.6642,   0.8225,   3.0309,  ...,   2.8935,   1.3534,  -2.4097],
         [  0.1253,  18.2364, -11.7023,  ...,   0.7288,   8.2506,   4.2057],
         [ -1.8094,  16.9949, -11.4810,  ...,  20.3952,   8.1938,  10.0478],
         ...,
         [ -1.6447,  -9.3096,   0.2799,  ...,  -0.5513,   6.1136,   1.5820],
         [ -3.6652,  -2.5033,  -2.4572,  ...,  -3.2551,  -3.9572,  -3.1280],
         [ -5.4513,  14.5782,   0.0000,  ...,  19.8965,  -0.6447,   6.1819]]],
       device='cuda:0', grad_fn=<AddBackward0>)

In [12]:
# Using greedy decoding to determine the word piece.


def compute_TruCLeS():
    text = convert_text_to_tokens(test_dataset[0][2], bpe_model_path, tokens_path)
    print(text)
    
    return

In [14]:
compute_TruCLeS()

tensor([346, 346, 346, 346, 346, 346, 242, 346, 346, 346, 270,   3, 346, 346,
         72,  22,   3, 346, 346,  72, 270, 346, 346, 346, 346, 216,  22, 346,
        346, 346, 346, 105, 346, 346, 346, 346,  48, 346, 346,  22,  48, 346])


In [ ]:
from torch

class CEM_Model(torch.nn.Module)
    